In [81]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

%matplotlib inline

In [82]:
import lightgbm as lgb

In [83]:
df = pd.read_csv('data/jiebao_all_user_processed.csv')
df.head()

,Unnamed: 0,user_id,follow_count,fans_count,gender,location,level,regis_year,regis_month,regis_day,properties,mileage,car_like,excellent_post_count,all_post_count,label
0,0,oden123,0,0,2,1,1,2018,3,3,1,4430.0,1,0,0,True
1,1,生活1934626,2,4,2,2,1,2012,5,5,1,4270.0,2,0,0,True
2,2,wangzi1125,0,1,2,3,1,2017,6,6,1,2090.0,3,0,1,True
3,3,房产专家谢广财,3,2,2,4,1,2014,2,2,1,1120.0,4,0,0,True
4,4,南宫晗笑,4,5,2,5,1,2017,7,7,1,740.0,1,0,14,True


In [84]:
df.keys()

Index(['Unnamed: 0', 'user_id', 'follow_count', 'fans_count', 'gender',
       'location', 'level', 'regis_year', 'regis_month', 'regis_day',
       'properties', 'mileage', 'car_like', 'excellent_post_count',
       'all_post_count', 'label'],
      dtype='object')

In [85]:
df.dtypes

Unnamed: 0                int64
user_id                  object
follow_count              int64
fans_count                int64
gender                    int64
location                  int64
level                     int64
regis_year                int64
regis_month               int64
regis_day                 int64
properties                int64
mileage                 float64
car_like                  int64
excellent_post_count      int64
all_post_count            int64
label                      bool
dtype: object

In [86]:
x = df.drop(columns=['Unnamed: 0','user_id', 'label'])
x.head()

,follow_count,fans_count,gender,location,level,regis_year,regis_month,regis_day,properties,mileage,car_like,excellent_post_count,all_post_count
0,0,0,2,1,1,2018,3,3,1,4430.0,1,0,0
1,2,4,2,2,1,2012,5,5,1,4270.0,2,0,0
2,0,1,2,3,1,2017,6,6,1,2090.0,3,0,1
3,3,2,2,4,1,2014,2,2,1,1120.0,4,0,0
4,4,5,2,5,1,2017,7,7,1,740.0,1,0,14


In [87]:
y = df['label'].astype('uint8')
y.describe()

count    2961.000000
mean        0.083755
std         0.277068
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: label, dtype: float64

In [88]:
x, y = np.array(x).astype('int'), np.array(y).astype('int')
x.shape, y.shape

((2961, 13), (2961,))

### Split dataset into train/validation/test set

In [91]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,  random_state = 42)

x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5,  random_state = 42)

In [92]:
#print(x_train.shape, y_train.shape)

#y_test.describe()

### Build SVM model then train it

In [93]:
clf = SVC(C=0.8, kernel='rbf', class_weight='balanced')

In [94]:
clf.fit(x_train, y_train)

SVC(C=0.8, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [95]:
pred = clf.predict(y_test)

ValueError: Expected 2D array, got 1D array instead:
array=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1

In [96]:
print(classification_report(y_true=y_test, y_pred=pred))

ValueError: Found input variables with inconsistent numbers of samples: [445, 444]

### Build lightgbm model and train it 

#### Dataset construction based on numpy data

In [104]:
x_train[0], y_train[0]

(array([   1,    0,    2,    9,    1, 2016,    2,    2,    1, 5170,   87,
           0,    0]), 0)

In [105]:
train_data = lgb.Dataset(x_train, label=y_train)
val_data = lgb.Dataset(x_val, label=y_val)
test_data = lgb.Dataset(x_test, label = y_test)

In [106]:
train_data = lgb.Dataset(x_train, 
                         label=y_train,
                         feature_name=['follow_count', 'fans_count', 'gender',
       'location', 'level', 'regis_year', 'regis_month', 'regis_day',
       'properties', 'mileage', 'car_like', 'excellent_post_count',
       'all_post_count',],
                         categorical_feature=['gender',
       'location', 'level', 'regis_year', 'regis_month', 'regis_day',
       'properties', 'car_like', ],
                        )

#### Setting parameters

In [107]:
param = {'num_leaves': 30,
        'num_trees': 10,
        'objective':'binary'}
param['metric'] = 'auc'

#### Training 

In [108]:
num_round = 100
bst = lgb.train(param, 
                train_data,
                num_round,
                valid_sets=[val_data]
               )

[1]	valid_0's auc: 0.678474
[2]	valid_0's auc: 0.632908
[3]	valid_0's auc: 0.635727
[4]	valid_0's auc: 0.633167
[5]	valid_0's auc: 0.638936
[6]	valid_0's auc: 0.641755
[7]	valid_0's auc: 0.646617
[8]	valid_0's auc: 0.634463
[9]	valid_0's auc: 0.63025
[10]	valid_0's auc: 0.634852


/Users/pierre_pc/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/pierre_pc/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1040: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/pierre_pc/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1042: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['car_like', 'gender', 'level', 'location', 'properties', 'regis_day', 'regis_month', 'regis_year']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/Users/pierre_pc/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:685: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_fe

In [109]:
# save model
json_model = bst.dump_model()

#### Prediction

In [114]:
y_pred = bst.predict(x_test)

In [117]:
y_pred

array([0.137172  , 0.05624423, 0.11289454, 0.06069188, 0.05275511,
       0.20077362, 0.04533636, 0.0376026 , 0.07657854, 0.08667259,
       0.06334421, 0.13569659, 0.04917838, 0.06069188, 0.03949231,
       0.08368596, 0.03797063, 0.03572493, 0.08281547, 0.0647533 ,
       0.04838748, 0.03916557, 0.04754473, 0.03967516, 0.07507997,
       0.24188647, 0.11951405, 0.25317818, 0.05442253, 0.1233533 ,
       0.18301084, 0.05789519, 0.03713245, 0.03713245, 0.05634898,
       0.04428905, 0.03819362, 0.1312443 , 0.04357616, 0.05366682,
       0.05159922, 0.05010155, 0.04944054, 0.14108052, 0.10014151,
       0.04867864, 0.04561091, 0.04183488, 0.03867161, 0.06040533,
       0.05398911, 0.04009292, 0.04229206, 0.04161392, 0.06741955,
       0.21404314, 0.04959207, 0.05610745, 0.14339137, 0.0376216 ,
       0.03632822, 0.19393053, 0.25317818, 0.03703375, 0.07585979,
       0.03537826, 0.07702664, 0.05338236, 0.09416562, 0.10471395,
       0.03797063, 0.05925651, 0.1953484 , 0.04846116, 0.04291

In [121]:
y_pred = y_pred >= 0.5

In [122]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(accuracy_score(y_test, y_pred))

0.9146067415730337


In [123]:
print(classification_report(y_true=y_test, y_pred=y_pred))

             precision    recall  f1-score   support

          0       0.91      1.00      0.96       407
          1       0.00      0.00      0.00        38

avg / total       0.84      0.91      0.87       445



/Users/pierre_pc/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
